In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas
import numpy
import numpy.linalg

import lib.utils as utils

In [3]:
timus = pandas.read_csv('../data/timus.csv', index_col=0)

/home/ec2-user/mount_point/tfnewenv/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
filtered_timus = utils.filter_timus_by_unique_accepted(timus, lambda cnt: cnt > 100)

In [162]:
def suggest(timus,
            top_count,
            k=200,
           ):
    solved_matrix, authors_encoder, problems_encoder = utils.build_solved_matrix(timus=timus, enable_negative=True)
    U, S, VT = numpy.linalg.svd(solved_matrix, full_matrices=False)
    A = (U * S)[:,:k]
    B = VT[:k,:]
    C = A.dot(B)
    suggestions = dict()
    for author_id in range(authors_encoder.get_set_size()):
        user_suggestions = []
        for problem_id in range(problems_encoder.get_set_size()):
            if solved_matrix[author_id][problem_id] == 1:
                continue
            real_problem_id = problems_encoder.decode(problem_id)
            user_suggestions.append(
                utils.SingleSuggest(
                    problem_index=real_problem_id,
                    score=C[author_id][problem_id]
                )
            )
        real_author_id = authors_encoder.decode(author_id)
        suggestions[real_author_id] = sorted(user_suggestions, key=lambda s: s.score, reverse=True)[:top_count]
    return suggestions

In [163]:
svd_based_scores = utils.get_algorithm_scores(
    timus=filtered_timus,
    suggest_for_timus=lambda t: suggest(timus=t, top_count=5),
)

In [164]:
svd_based_scores

array([0.09347443, 0.09370988, 0.04859611, 0.06557377, 0.06360424,
       0.06374172, 0.04435798, 0.06997743, 0.06642336, 0.07320644,
       0.06546763, 0.09397944])

In [165]:
pandas.Series(svd_based_scores).describe()

count    12.000000
mean      0.070176
std       0.016367
min       0.044358
25%       0.063707
50%       0.065999
75%       0.078273
max       0.093979
dtype: float64

In [7]:
svd_based_scores

array([0.08289242, 0.09884467, 0.05939525, 0.06557377, 0.07773852,
       0.06870861, 0.05992218, 0.07298721, 0.0729927 , 0.08052709,
       0.07697842, 0.10279001])

In [8]:
pandas.Series(svd_based_scores).describe()

count    12.000000
mean      0.076613
std       0.013555
min       0.059395
25%       0.067925
50%       0.074986
75%       0.081118
max       0.102790
dtype: float64

In [159]:
solved_matrix, authors_encoder, problems_encoder = utils.build_solved_matrix(timus=filtered_timus, enable_negative=True)

In [160]:
x = solved_matrix[authors_encoder.encode(99162)]

In [161]:
x[x < 0]

array([-0.09966799, -0.09966799, -0.04995837, -0.04995837, -0.19737532,
       -0.71629787, -0.29131261, -0.04995837, -0.24491866, -0.04995837])

In [85]:
numpy.zeros(dtype=float)

TypeError: Required argument 'shape' (pos 1) not found

In [78]:
len(set(filtered_timus['author_id']))

3247

In [84]:
def accepted_filter_per_author(cnt):
    return cnt > 100

In [94]:
def filter_timus_by_unique_accepted(timus,
                                    accepted_filter_per_author
                                   ):
    return (
        timus
            .groupby('author_id')
            .filter(
                lambda group: accepted_filter_per_author(len(set(group[group['judgement_result'] == 'Accepted']['problem_id'])))
            )
    )

In [95]:
len(set(filter_timus_by_unique_accepted(timus, accepted_filter_per_author)['author_id']))

3247

In [9]:
import scipy.special

In [60]:
f(-20)

-0.7615941559557646

In [158]:
import imp
imp.reload(utils)

<module 'lib.utils' from '../lib/utils.py'>

In [71]:
filtered_timus.head()

,author_id,date,execution_time,judgement_result,language,memory_used,problem_id,submit_id,test_number
18,10023,2000-09-26 19:28:53,0.020,Accepted,FreePascal 1.0.6,121,1000,291,0
32,10023,2000-09-26 20:53:03,0.030,Wrong answer,FreePascal 1.0.6,149,1004,308,1
33,10023,2000-09-26 20:57:00,0.020,Wrong answer,FreePascal 1.0.6,149,1004,309,1
73,10023,2000-09-27 13:20:00,0.120,Wrong answer,FreePascal 1.0.6,657,1004,352,1
74,10023,2000-09-27 13:40:00,0.015,Wrong answer,FreePascal 1.0.6,125,1005,353,1
